# IMPORTS

In [1]:
import torch, torchaudio

C:\Users\leona\anaconda3\envs\reprodl\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')


In [ ]:
from torch import nn
from torch.nn import functional as F

In [2]:
import pytorch_lightning as pl

In [27]:
pip install torchaudio

Note: you may need to restart the kernel to use updated packages.


In [37]:
pip install PySoundFile

Note: you may need to restart the kernel to use updated packages.


In [3]:
torch.cuda.is_available()

False

In [4]:
# thus use smaller parameters, smaller number of filters

In [5]:
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

# Step 1: loading the data

In [7]:

datapath = Path('DATA\ESC50')

In [8]:
datapath.exists()

True

In [9]:
csv = pd.read_csv(datapath / Path('meta/esc50.csv'))

In [10]:
csv

,filename,fold,target,category,esc10,src_file,take
0,1-100032-A-0.wav,1,0,dog,True,100032,A
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A
...,...,...,...,...,...,...,...
1995,5-263831-B-6.wav,5,6,hen,False,263831,B
1996,5-263902-A-36.wav,5,36,vacuum_cleaner,False,263902,A
1997,5-51149-A-25.wav,5,25,footsteps,False,51149,A
1998,5-61635-A-8.wav,5,8,sheep,False,61635,A


In [21]:
x, sr = torchaudio.load(datapath/'audio'/csv.iloc[0, 0], normalize = True)

TypeError: Invalid file: WindowsPath('DATA/ESC50/audio/1-100032-A-0.wav')

In [40]:
x.shape

NameError: name 'x' is not defined

In [ ]:
torchaudio.transforms.Resample(orig_freq = sr, new_freq = 8000)

In [ ]:
h = torchaudio.transforms.MelSpectogram(sample_rate = sr)(x)

In [ ]:
plt.imshow(h[0])

In [ ]:
h = torchaudio.transforms.AmplitudeToDB()(h)

In [ ]:
plt.imshow(h[0])

# Data loader

In [ ]:
class ESC50Dataset(torch.utils.data.Dataset):
    def __init__(self, path: Path = Path(('DATA/ESC50'),
                 sample_rate :int = 8000,
                 folds = [1]):
        #Load csv filed and initialize torchaudio.trasnforms
        #Resample __>  MelSpectogram --> AmplitudeToDB
        self.path = path
        self.csv = pd.read_csv(datapath / Path('meta/esc50.csv'))
        self.csv = self.csv[self.csv['fold'].isin(folds)]
        self.resample = torchaudio.transforms.Resample(orig_freq = 44100,
                        new_freq= sample_rate)
        self.melspec = torchaudio.transforms.MelSpectrogram(sample_rate = sample_rate)
        self.db = torchaudio.transforms.AmplitudeToDB()
                 
    def __getitem__(self, index):
        #returns(xb, yb) pairs
        row = self.csv-iloc[index]
        wav, _ = torchaudio.load(self.path / 'audio' / row['filename'])
        label = row['target']
        xb = self.db(self.melspec(self.resample(wav)))
        return xb, label
                 
    def __len__(self):
        #returns length
        return len(self.csv)

                
    

In [ ]:
train_data = ESC50Dataset(folds = [1])

In [ ]:
for xb, yb in train_data:
    break

In [ ]:
xb.shape

In [ ]:
train_data = ESC50Dataset(fold = [1])
val_data = ESC50Dataset(fold = [2])
test_data = ESC50Dataset(fold = [3])

In [ ]:
torch.utils.data.DataLoader(train_data, batch_size = 8, shuffle = True)

In [ ]:
train_loader = \
    torch.utils.data.DataLoader(train_data, batch_size = 8, shuffle = True)

In [ ]:
test_loader = \
    torch.utils.data.DataLoader(test_data, batch_size = 8, shuffle = True)

In [ ]:
val_loader = \
    torch.utils.data.DataLoader(val_data, batch_size = 8, shuffle = True)

# building the MODEL

In [ ]:
class AudioNet(pl.LigthningModel):
 
    def __init__(self, n_classes = 50, base_filters = 64):
        super().__init__()
        self.conv1 = nn.Conv2d(1, base_filters, 11, padding=5)
        self.bn1 = nn.BatchNorm2d(base_filters)
        self.conv2 = nn.Conv2d(base_filters, base_filters, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(base_filters)
        self.pool1 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(base_filters, base_filters * 2, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(base_filters * 2)
        self.conv4 = nn.Conv2d(base_filters * 2, base_filters * 4, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(base_filters * 4)
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(base_filters * 4, n_classes)
 
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool1(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool2(x)
        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = self.fc1(x[:, :, 0, 0])
        return x
    def training_step(self, batch, batch_idx):
        # training_step defined the train loop.
        # It is independent of forward
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, x)
        # Logging to TensorBoard by default
        self.log('train_loss', loss, on_step = True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [ ]:
pl.seed_Everything(0)

In [ ]:
audionet = AudioNet()

In [ ]:
xb, yb = next(iter(train_loader))

In [ ]:
xb.shape

In [ ]:
audionet(xb).shape

# Train

In [ ]:
trainer = pl.Trainer(max_epochs = 1) #gpus = 1

In [ ]:
trainer.fit(audionet, train_loader, val_loader)

In [ ]:
trainer.test(audionet, test_loader) #test set here